In [1]:
import json
import os

import numpy as np
import pandas as pd
import requests
from openai import AzureOpenAI
from tqdm import tqdm

In [2]:
news_df = pd.read_csv("../../data/processed/6-23_to_7-21_nvda_news.csv")
stock_df = pd.read_csv("../../data/processed/6-23_to_7-21_nvda_stock.csv")

In [3]:
news_df.sort_values(by="assigned_date", inplace=True)
news_df.reset_index(drop=True, inplace=True)
news_df

,datetime,headline,summary,assigned_date
0,2025-06-23 11:37:18,AI Chips Today - AI Intelligence Insights Expa...,BCC Research has recently launched a specializ...,2025-06-23
1,2025-06-23 13:45:00,Will Nebius Achieve its $1B ARR and Up to $700...,"NBIS posts 385% Q1 revenue growth, eyes up to ...",2025-06-23
2,2025-06-23 13:00:00,Siemens turbocharges semiconductor and PCB des...,"At the 2025 Design Automation Conference, Siem...",2025-06-23
3,2025-06-23 12:53:00,AMD Stock Gets New Buy Rating As AI Efforts Ta...,Advanced Micro Devices has turned a Wall Stree...,2025-06-23
4,2025-06-23 11:49:34,"Nvidia, Other Tech Stocks Likely to Shake Off ...",Nvidia (NVDA) and other technology stocks will...,2025-06-23
...,...,...,...,...
1205,2025-07-20 20:55:27,"Stocks Going On Sale, Netflix Fell After Earni...",I see high-quality growth stocks like NFLX and...,2025-07-21
1206,2025-07-20 20:37:00,The stock market is being led by a new group o...,There's a new group of big winners emerging.,2025-07-21
1207,2025-07-20 16:33:00,AMD's quiet move may shock gaming GPU market,The company's new plans may allow it to take t...,2025-07-21
1208,2025-07-20 23:05:00,The Smartest Artificial Intelligence (AI) Stoc...,AI hardware investments are top picks due to t...,2025-07-21


In [4]:
stock_df.head()

,date,price
0,2025-06-23,144.169998
1,2025-06-24,147.899994
2,2025-06-25,154.309998
3,2025-06-26,155.020004
4,2025-06-27,157.750000


In [5]:
stock_df.tail()

,date,price
15,2025-07-15,170.699997
16,2025-07-16,171.369995
17,2025-07-17,173.000000
18,2025-07-18,172.410004
19,2025-07-21,171.380005


In [6]:
token = os.getenv("TOKEN")
subscription = os.getenv("SUBSCRIPTION_ID")
resource_group = os.getenv("RESOURCE_GROUP")
resource_name = os.getenv("RESOURCE_NAME")
model_deployment_name = "gpt41-mini-news-sentiment"

In [7]:
deploy_params = {"api-version": "2025-04-01-preview"}
deploy_headers = {
    "Authorization": "Bearer {}".format(token),
    "Content-Type": "application/json",
}

In [8]:
deploy_data = {
    "sku": {"name": "developertier", "capacity": 70},
    "properties": {
        "model": {
            "format": "OpenAI",
            "name": "gpt-4.1-mini-2025-04-14.ft-9e2dec8fc72c499da71bc5668f52a1be-001",
            "version": "1",
        }
    },
}
deploy_data = json.dumps(deploy_data)

In [9]:
request_url = f"https://management.azure.com/subscriptions/{subscription}/resourceGroups/{resource_group}/providers/Microsoft.CognitiveServices/accounts/{resource_name}/deployments/{model_deployment_name}"

print("Creating a new deployment...")

r = requests.put(
    request_url, params=deploy_params, headers=deploy_headers, data=deploy_data
)

print(r)
print(r.reason)
print(r.json())

Creating a new deployment...
<Response [201]>
Created
{'id': '/subscriptions/b5a4f554-5572-486f-8de8-7624c85141b4/resourceGroups/SentimentLLM-UTD-Sweden/providers/Microsoft.CognitiveServices/accounts/SentimentLLM-UTD-Sweden/deployments/gpt41-mini-news-sentiment', 'type': 'Microsoft.CognitiveServices/accounts/deployments', 'name': 'gpt41-mini-news-sentiment', 'sku': {'name': 'developertier', 'capacity': 70}, 'properties': {'model': {'format': 'OpenAI', 'name': 'gpt-4.1-mini-2025-04-14.ft-9e2dec8fc72c499da71bc5668f52a1be-001', 'version': '1'}, 'versionUpgradeOption': 'NoAutoUpgrade', 'currentCapacity': 70, 'capabilities': {'chatCompletion': 'true', 'area': 'EUR', 'responses': 'true', 'assistants': 'true'}, 'provisioningState': 'Creating', 'rateLimits': [{'key': 'request', 'renewalPeriod': 60, 'count': 70}, {'key': 'token', 'renewalPeriod': 60, 'count': 70000}]}, 'systemData': {'createdBy': 'Priyanath_Maji@bcbstx.com', 'createdByType': 'User', 'createdAt': '2025-07-23T15:32:18.5599594Z', 

In [10]:
system_prompt = {
    "role": "system",
    "content": "You are a financial sentiment classifier. Respond with one word: neutral, positive, or negative.",
}

sentiment_to_score = {"positive": 1.0, "neutral": 0.0, "negative": -1.0}
sentiment_scores, sentiment_labels = [], []

In [12]:
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),
    api_key=os.getenv("AZURE_API_KEY"),
    api_version="2024-12-01-preview",
)

In [20]:
for summary in tqdm(news_df["summary"].tolist()[358:], desc="Classifying sentiment"):
    response = client.chat.completions.create(
        model=model_deployment_name,
        messages=[
            system_prompt,
            {
                "role": "user",
                "content": f"Classify the sentiment of this sentence: {summary}",
            },
        ],
        max_completion_tokens=1,
    )
    sentiment = response.choices[0].message.content.strip().lower()

    sentiment_labels.append(sentiment)
    sentiment_scores.append(sentiment_to_score.get(sentiment, 0.0))

Classifying sentiment: 100%|██████████| 852/852 [12:25<00:00,  1.14it/s]  


In [21]:
with open("../../models/gpt-4.1/gpt-4.1-mini-ft-news-sentiment.txt", "w") as f:
    for label in sentiment_labels:
        f.write(label + "\n")

In [27]:
all_dates = pd.date_range(start="2025-06-23", end="2025-07-21").date
all_dates[:5]

array([datetime.date(2025, 6, 23), datetime.date(2025, 6, 24),
       datetime.date(2025, 6, 25), datetime.date(2025, 6, 26),
       datetime.date(2025, 6, 27)], dtype=object)

In [23]:
news_df["sentiment_label"] = sentiment_labels
news_df["sentiment_score"] = sentiment_scores

In [34]:
news_df.head()

,datetime,headline,summary,assigned_date,sentiment_label,sentiment_score
0,2025-06-23 11:37:18,AI Chips Today - AI Intelligence Insights Expa...,BCC Research has recently launched a specializ...,2025-06-23,neutral,0.0
1,2025-06-23 13:45:00,Will Nebius Achieve its $1B ARR and Up to $700...,"NBIS posts 385% Q1 revenue growth, eyes up to ...",2025-06-23,positive,1.0
2,2025-06-23 13:00:00,Siemens turbocharges semiconductor and PCB des...,"At the 2025 Design Automation Conference, Siem...",2025-06-23,positive,1.0
3,2025-06-23 12:53:00,AMD Stock Gets New Buy Rating As AI Efforts Ta...,Advanced Micro Devices has turned a Wall Stree...,2025-06-23,positive,1.0
4,2025-06-23 11:49:34,"Nvidia, Other Tech Stocks Likely to Shake Off ...",Nvidia (NVDA) and other technology stocks will...,2025-06-23,neutral,0.0


In [35]:
daily_sentiment = (
    news_df.groupby("assigned_date")["sentiment_score"].mean().reset_index()
)
daily_sentiment.columns = ["date", "avg_sentiment"]

In [43]:
daily_sentiment.head()

,date,avg_sentiment
0,2025-06-23,0.242424
1,2025-06-24,0.419355
2,2025-06-25,0.200000
3,2025-06-26,0.551724
4,2025-06-27,0.341463


In [44]:
stock_df.head()

,date,price
0,2025-06-23,144.169998
1,2025-06-24,147.899994
2,2025-06-25,154.309998
3,2025-06-26,155.020004
4,2025-06-27,157.750000


In [45]:
merged_df = pd.merge(daily_sentiment, stock_df, on="date", how="left")

In [51]:
merged_df.ffill(inplace=True)

In [52]:
merged_df

,date,avg_sentiment,price
0,2025-06-23,0.242424,144.169998
1,2025-06-24,0.419355,147.899994
2,2025-06-25,0.200000,154.309998
3,2025-06-26,0.551724,155.020004
4,2025-06-27,0.341463,157.750000
5,2025-06-28,0.526316,157.750000
6,2025-06-29,0.250000,157.750000
7,2025-06-30,0.527778,157.990005
8,2025-07-01,0.457627,153.300003
9,2025-07-02,0.411765,157.250000


In [53]:
merged_df.to_csv("../../data/processed/prices_and_sentiments.csv")